In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time

import os

# Data Capping

In [3]:
start_time = time.time()

directory_path = '../data/pems___101_N_Sep_2024/'
file_name_list = [f for f in os.listdir(directory_path) \
                  if os.path.isfile(os.path.join(directory_path, f)) and f[-1] == 'x' and f[0] != "~"]
file_name_list.sort()

thresh_lower = 0.1

# df = pd.read_excel(directory_path + file_name)
df_data_filtered = pd.read_csv(directory_path + 'data_filtered.csv', index_col=0)

dict_data_capped = {}

times_dict = {}
dict_data_capped["Times / Type"] = ["Type", "Number (data points)", "Mean (capped flow)", "Variance (capped flow)"]

for column_name in list(df_data_filtered.columns):
    print("column_name:", column_name)
    
    vals_list_str = df_data_filtered[column_name].tolist()
    
#     print("vals_list_str:", vals_list_str)
    
    dict_data_capped[column_name] = []
    dict_data_capped[column_name] += [vals_list_str[0]]
    
    vals_list_str = vals_list_str[1:]
    vals_list = [float(val) for val in vals_list_str]
    
    max_val = max(vals_list)
    
#     print("vals_list:", vals_list)
#     print("max_val:", max_val)
    
    vals_array_capped = np.array([val for val in vals_list if val != 0.0 and val >= max_val * thresh_lower])
    
#     print()
#     print("vals_list:", vals_list)
#     print()
#     print("vals_array_capped:", vals_array_capped)
    
    dict_data_capped[column_name] += [vals_array_capped.shape[0]]
    dict_data_capped[column_name] += [np.mean(vals_array_capped)]
    dict_data_capped[column_name] += [np.std(vals_array_capped)]
    print("dict_data_capped[column_name]:", dict_data_capped[column_name])
    print()


df_dict_data_capped = pd.DataFrame(dict_data_capped)
df_dict_data_capped.to_csv(directory_path + 'data_capped.csv', index = False)

end_time = time.time()

print()
print("Time:", end_time - start_time)



column_name: 001___402376_Palo_Alto___main.xlsx
vals_list_str: ['main', '3258', '4324', '4520', '4025', '3506', '3300', '3159', '3331', '4228', '4318', '3837', '3518', '3422', '3125', '3253', '4243', '4455', '4202', '3663', '3296', '3435', '3103', '4053', '3998', '3775', '3342', '3287', '3325', '4039', '4944', '5117', '4760', '4195', '4008', '3966', '3391', '4452', '4653', '4150', '3718', '3407', '3098', '3026', '3996', '4518', '4005', '3342', '3517', '3229', '240', '240', '240', '240', '240', '240', '240', '3800', '4706', '4612', '4146', '4000', '4024', '4065', '3886', '4681', '4786', '4430', '3941', '3680', '3602', '3502', '4460', '4688', '4229', '3461', '3037', '3188', '3574', '4488', '4606', '3984', '3608', '3216', '3158', '4143', '4289', '4595', '4272', '3658', '3419', '3163', '3931', '4674', '4853', '4434', '4072', '4132', '4251', '3355', '4172', '4207', '3700', '3267', '2980', '2713', '3446', '4273', '3892', '3947', '3062', '3461', '3067', '3406', '4241', '4119', '3305', '3498',

In [ ]:
# directory_path = '../data/pems___101_N_Sep_2024/'
# file_name_list = [f for f in os.listdir(directory_path) \
#                   if os.path.isfile(os.path.join(directory_path, f)) and f[-1] == 'x']
# file_name_list.sort()

# file_name_list



# Data Consolidating

# Data Consistency Enforcement

# Scratch Work

In [ ]:
["Type", "Mean flow", "Variance of flow"]